In [19]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pickle

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 700)

This notebook creates temperature and humidity features.

In [20]:
# import subset master dataset data
data_df = pd.read_pickle('./../data/gelderman/gelderman_master_dataset.pkl')
print(data_df.shape)
display(data_df.head())
display(data_df.info())

(174, 17)


,new_id,old_id,img_path,img_head,correct_img_date,date_placed_ARF,PMI_days,sex,age_at_death,est_stature_inches,est_weight,true_SOD_G_head,donor_date,img_torso,true_SOD_G_torso,img_limbs,true_SOD_G_limbs
0,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/004/00400124.12.JPG,00400124.12.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.00,180.0,3.0,00400124,00400124.07.JPG,3.0,00400124.10.JPG,3.0
1,006,UT08-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/006/00600212.07.JPG,00600212.07.JPG,2018-02-12,2018-01-09,34.0,1.0,81.0,NaN,NaN,3.0,00600212,00600212.25.JPG,3.0,00600212.06.JPG,3.0
2,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00525.08.JPG,00b00525.08.JPG,2016-05-25,2016-03-07,79.0,1.0,38.0,185.42,516.0,5.0,00b00525,00b00525.04.JPG,5.0,00b00525.27.JPG,3.0
3,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00818.07.JPG,00b00818.07.JPG,2016-08-18,2016-03-07,164.0,1.0,38.0,185.42,516.0,6.0,00b00818,00b00818.05.JPG,6.0,00b00818.11.JPG,3.0
4,00d,UT103-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00d/00d10116.07.JPG,00d10116.07.JPG,2019-01-16,2018-11-27,50.0,1.0,73.0,185.42,235.0,5.0,00d10116,00d10116.04.JPG,4.0,00d10116.06.JPG,4.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 173
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   new_id              174 non-null    object        
 1   old_id              174 non-null    object        
 2   img_path            174 non-null    object        
 3   img_head            174 non-null    object        
 4   correct_img_date    174 non-null    datetime64[ns]
 5   date_placed_ARF     174 non-null    datetime64[ns]
 6   PMI_days            174 non-null    float64       
 7   sex                 174 non-null    float64       
 8   age_at_death        174 non-null    float64       
 9   est_stature_inches  168 non-null    float64       
 10  est_weight          162 non-null    float64       
 11  true_SOD_G_head     174 non-null    float64       
 12  donor_date          174 non-null    object        
 13  img_torso           174 non-null    object        

None

In [21]:
# import LCD weather data (hourly)
weather_df = pd.read_pickle('/data/anau/temp_humidity_data/data/LCD/lcd_daily_avg.pkl')
print(weather_df.shape)
display(weather_df.head())
display(weather_df.info())

(4473, 3)


,date,HourlyDryBulbTemperature,HourlyRelativeHumidity
0,2011-01-01,11.472727,91.803030
1,2011-01-02,3.828571,64.628571
2,2011-01-03,-0.068750,57.812500
3,2011-01-04,3.084375,60.156250
4,2011-01-05,1.008333,83.216667


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4473 entries, 0 to 4472
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date                      4473 non-null   datetime64[ns]
 1   HourlyDryBulbTemperature  4473 non-null   float64       
 2   HourlyRelativeHumidity    4473 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 139.8 KB


None

In [22]:
subset_dict = data_df.to_dict('records')
row_counter = 0
# for each image
for i,row in enumerate(subset_dict):
    print('iter:', i, 'ID:', row['new_id'])
    img_date = row['correct_img_date']
    #print(img_date)
    
    time_windows = [(1,3), (4,7), (8,21), (22,56), (57,154)]
    
    for t in time_windows:
        # get start and end date of time window
        start = img_date - datetime.timedelta(days=t[0])
        end = img_date - datetime.timedelta(days=t[1])
        #print(start, end)
        
        # get weather data (temp and humidity) for time window
        mask = (weather_df['date'] <= start) & (weather_df['date'] >= end)
        subset_weather_df = weather_df.loc[mask].copy()
        #display(subset_weather_df.loc[mask])
        
        # aggregate weather data
        agg_df = subset_weather_df[['HourlyDryBulbTemperature', 'HourlyRelativeHumidity']].agg(['mean', 'std']).copy()
        #display(agg_df)
        temp_mean = round(agg_df.loc['mean', 'HourlyDryBulbTemperature'], 3)
        temp_std = round(agg_df.loc['std', 'HourlyDryBulbTemperature'], 3)
        hum_mean = round(agg_df.loc['mean', 'HourlyRelativeHumidity'], 3)
        hum_std = round(agg_df.loc['std', 'HourlyRelativeHumidity'], 3)
        
        # add to dict
        time_window_s = str(t[0]) + '_' + str(t[1])

        row['temp'+'_'+time_window_s+'_mean'] = temp_mean
        row['temp'+'_'+time_window_s+'_std'] = temp_std

        row['hum'+'_'+time_window_s+'_mean'] = hum_mean
        row['hum'+'_'+time_window_s+'_std'] = hum_std
        
        #print(row)
        #print()
        
    #break

    print()

# convert dict to dataframe
data_w_weather_df = pd.DataFrame.from_dict(subset_dict)

iter: 0 ID: 004

iter: 1 ID: 006

iter: 2 ID: 00b

iter: 3 ID: 00b

iter: 4 ID: 00d

iter: 5 ID: 011

iter: 6 ID: 01c

iter: 7 ID: 01e

iter: 8 ID: 02a

iter: 9 ID: 02d

iter: 10 ID: 02d

iter: 11 ID: 044

iter: 12 ID: 044

iter: 13 ID: 059

iter: 14 ID: 059

iter: 15 ID: 06b

iter: 16 ID: 076

iter: 17 ID: 07b

iter: 18 ID: 07b

iter: 19 ID: 07b

iter: 20 ID: 07c

iter: 21 ID: 07c

iter: 22 ID: 082

iter: 23 ID: 08f

iter: 24 ID: 090

iter: 25 ID: 091

iter: 26 ID: 095

iter: 27 ID: 097

iter: 28 ID: 097

iter: 29 ID: 09b

iter: 30 ID: 0a5

iter: 31 ID: 0ae

iter: 32 ID: 0b2

iter: 33 ID: 0b9

iter: 34 ID: 0b9

iter: 35 ID: 0c2

iter: 36 ID: 0d9

iter: 37 ID: 0e8

iter: 38 ID: 0f2

iter: 39 ID: 0f2

iter: 40 ID: 12a

iter: 41 ID: 12b

iter: 42 ID: 12b

iter: 43 ID: 137

iter: 44 ID: 139

iter: 45 ID: 13c

iter: 46 ID: 153

iter: 47 ID: 169

iter: 48 ID: 16c

iter: 49 ID: 18c

iter: 50 ID: 197

iter: 51 ID: 1bc

iter: 52 ID: 1db

iter: 53 ID: 1e3

iter: 54 ID: 1f7

iter: 55 ID: 215

it

In [23]:
data_w_weather_df.head()

,new_id,old_id,img_path,img_head,correct_img_date,date_placed_ARF,PMI_days,sex,age_at_death,est_stature_inches,est_weight,true_SOD_G_head,donor_date,img_torso,true_SOD_G_torso,img_limbs,true_SOD_G_limbs,temp_1_3_mean,temp_1_3_std,hum_1_3_mean,hum_1_3_std,temp_4_7_mean,temp_4_7_std,hum_4_7_mean,hum_4_7_std,temp_8_21_mean,temp_8_21_std,hum_8_21_mean,hum_8_21_std,temp_22_56_mean,temp_22_56_std,hum_22_56_mean,hum_22_56_std,temp_57_154_mean,temp_57_154_std,hum_57_154_mean,hum_57_154_std
0,004,UT06-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/004/00400124.12.JPG,00400124.12.JPG,2018-01-24,2018-01-09,15.0,0.0,77.0,64.00,180.0,3.0,00400124,00400124.07.JPG,3.0,00400124.10.JPG,3.0,9.315,0.888,71.621,12.679,-3.108,7.418,62.885,6.736,-0.699,6.638,74.493,16.490,4.205,5.889,78.049,14.677,16.426,6.113,76.979,10.014
1,006,UT08-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/006/00600212.07.JPG,00600212.07.JPG,2018-02-12,2018-01-09,34.0,1.0,81.0,NaN,NaN,3.0,00600212,00600212.25.JPG,3.0,00600212.06.JPG,3.0,9.461,4.403,86.695,15.567,3.024,1.901,80.516,12.758,4.897,3.853,69.225,14.323,0.878,7.154,75.502,15.670,13.500,7.154,78.095,10.514
2,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00525.08.JPG,00b00525.08.JPG,2016-05-25,2016-03-07,79.0,1.0,38.0,185.42,516.0,5.0,00b00525,00b00525.04.JPG,5.0,00b00525.27.JPG,3.0,19.485,0.935,57.198,2.203,18.725,0.697,76.769,8.882,17.617,4.230,64.476,8.287,16.590,4.368,58.561,15.086,7.484,7.232,69.266,15.024
3,00b,UT16-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00b/00b00818.07.JPG,00b00818.07.JPG,2016-08-18,2016-03-07,164.0,1.0,38.0,185.42,516.0,6.0,00b00818,00b00818.05.JPG,6.0,00b00818.11.JPG,3.0,27.174,1.174,71.959,6.483,28.471,0.317,63.152,0.949,26.955,1.031,72.737,6.045,26.864,1.680,67.719,7.432,18.606,5.250,63.088,13.413
4,00d,UT103-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/00d/00d10116.07.JPG,00d10116.07.JPG,2019-01-16,2018-11-27,50.0,1.0,73.0,185.42,235.0,5.0,00d10116,00d10116.04.JPG,4.0,00d10116.06.JPG,4.0,3.511,1.705,89.241,11.812,2.666,2.899,57.956,9.740,10.313,2.750,84.482,10.465,5.370,3.923,79.734,12.914,18.516,7.254,81.323,8.885


In [24]:
data_w_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   new_id              174 non-null    object        
 1   old_id              174 non-null    object        
 2   img_path            174 non-null    object        
 3   img_head            174 non-null    object        
 4   correct_img_date    174 non-null    datetime64[ns]
 5   date_placed_ARF     174 non-null    datetime64[ns]
 6   PMI_days            174 non-null    float64       
 7   sex                 174 non-null    float64       
 8   age_at_death        174 non-null    float64       
 9   est_stature_inches  168 non-null    float64       
 10  est_weight          162 non-null    float64       
 11  true_SOD_G_head     174 non-null    float64       
 12  donor_date          174 non-null    object        
 13  img_torso           174 non-null    object        

In [25]:
# write to pickle
data_w_weather_df.to_pickle('./../data/gelderman/gelderman_master_dataset_w_weather.pkl')